In [116]:
import pandas as pd 
import numpy as np
from collections import Counter
import gzip
import os
import subprocess

import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 
import subprocess

cell_type = 'HFF'
cell_type = 'HEK'
output_dir = '../output/fithic/{}'.format(cell_type)

## Analyzing bin counts 

In [117]:


def chrom_num(x):
    """
    Assign a chromosome number. 
    
    """
    if x not in ['chrX', 'chrY']:
        x = int(x.replace('chr', ''))
        
    elif x == 'chrX':
        x = 23
        
    elif x == 'chrY':
        x = 24
        
    return(x)

def chrom_pair_key(x):
    """
    Generate a chromsome key for sorting. 
    
    """
    
    return(chrom_num(x[0]), chrom_num(x[2]), x[1], x[3])


def chrom_key(x):
    """
    Generate a chromsome key for sorting. 
    
    """
    
    return(chrom_num(x[0]), x[1])


def decide_read_bin(pos, res):
    """
    Decide what bin to use for read with start end. 
    
    """
    
    cbin = int(np.floor(pos / res))
    
    return(cbin)

def decide_read_pair_bin(posA, posB, res):
    """
    Decide what bin to use for read pair. 
    
    """

    binA = decide_read_bin(posA, res)
    binB = decide_read_bin(posB, res)
    
    return([binA, binB])


def decide_read_pos(pos, res):
    """
    Decide what bin to use for read with start end. 
    
    """
    
    cbin = int(np.floor(pos / res) * res)
    
    return(cbin)

def decide_read_pair_pos(posA, posB, res):
    """
    Decide what bin to use for read pair. 
    
    """

    binA = decide_read_pos(posA, res)
    binB = decide_read_pos(posB, res)
    
    return([binA, binB])

In [118]:
read_pairs_fn = '/gpfs/data01/glasslab/home/joreyna/projects/CSE283/imarge_project/'
read_pairs_fn += 'data/iMARGI_seq/filter200k_final_{}_iMARGI.pairs'.format(cell_type)

res = 40000
interactions_fn = os.path.basename(read_pairs_fn)
interactions_fn = interactions_fn.replace('pairs', 'res{}.interactions.txt.gz'.format(res))
interactions_fn = os.path.join(output_dir, interactions_fn)
margin_counts_fn = interactions_fn.replace('interactions', 'margin_counts')

# Excluding non-human chromosomes 
chroms = list(range(1, 23)) + ['X', 'Y']
chroms = ['chr{}'.format(x) for x in chroms]

In [119]:
if not os.path.exists(interactions_fn):

    print('Analyzing interactions.')
    
    contact_map = Counter()
    margin_counts = Counter()
    
    with open(read_pairs_fn) as f:
        
        for i, line in enumerate(f):
            
            if line.startswith('#'):
                continue 
                
            line = line.strip()
            vals = line.split()
            chromA, posA, chromB, posB = vals[1:5]
            posA, posB = int(posA), int(posB)

            if chromA in chroms and chromB in chroms: 
                binA, binB = decide_read_pair_pos(posA, posB, res)
                contact_key = (chromA, binA, chromB, binB)
                contact_map[contact_key] += 1 

            if i % 1000000 == 0:
                print(i)
          

In [120]:
# Write the contact map in sorted order by chromsome and positions 
if not os.path.exists(interactions_fn):
    
    print('Writing interactions file.')
    
    # Getting and sorting contact keys 
    contact_keys = contact_map.keys()
    sorted_keys = sorted(contact_keys, key=chrom_pair_key)
    
    # Writing the interactions 
    with gzip.open(interactions_fn, 'wb') as f: 
        for key in sorted_keys: 
            counts = contact_map[key]
            msg = list(key) + [counts]
            msg = '\t'.join([str(x) for x in msg]) + '\n'
            f.write(msg.encode())    

In [121]:
# Write the contact map in sorted order by chromsome and positions 
if not os.path.exists(margin_counts_fn):
    
    print('Writing margin counts file.')
    
    # Getting and sorting contact keys 
    margin_keys = contact_map.keys()
    sorted_keys = sorted(contact_keys, key=chrom_key)
    
    # Writing the interactions 
    with gzip.open(margin_counts_fn, 'wb') as f: 
        for key in sorted_keys: 
            counts = contact_map[key]

            # Write the chr, dummy 0, midpoint, counts, and dummy 1.
            msg = [key[0], 0, key[1], counts, 0]
            msg = '\t'.join([str(x) for x in msg]) + '\n'
            f.write(msg.encode())

In [122]:
fithic_script = '/gpfs/data01/glasslab/home/joreyna/projects/CSE283/imarge_project/software/fithic/fithic/fithic.py'
python_path = '/home/joreyna/.conda/envs/tf_binding_nnet/bin/python'
sig_ints = os.path.join(output_dir, 'FitHiC.spline_pass1.res40000.significances.txt.gz')

if not os.path.exists(sig_ints):
    cmd = '{} {} -i {} -f {} -o {} -r {} -x All'.\
        format(python_path, fithic_script, interactions_fn, margin_counts_fn, output_dir, res)
    sp = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    sp.communicate()

    print(comm[0].decode())
    print(comm[1].decode())

## Running FitHiC 

In [123]:
flt_sig_ints = sig_ints.replace('gz', 'flt.gz')

if not os.path.exists(flt_sig_ints):

    print('Extracting significant interactions.')
    
    with gzip.open(sig_ints) as f, gzip.open(flt_sig_ints, 'wb') as fw:
        
        header = next(f) 
        fw.write(header)
        for i, line in enumerate(f):
                
            line = line.strip()
            vals = line.split()
            chromA, posA, chromB, posB, counts, pval, qval, bias1, bias2 = vals
            pval, qval = float(pval), float(qval)
            
            if qval < 0.05: 
                fw.write(line + '\n'.encode())

Extracting significant interactions.
